In [1]:
# Buňka 1: Importy
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

print("✅ Knihovny úspěšně naimportovány.")

/home/luk-hrd/miniconda3/envs/mistral-lora/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Knihovny úspěšně naimportovány.


In [2]:
# Buňka 2: Konfigurace
# Použijeme přímo identifikátor modelu z Hugging Face Hub
base_model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# --- Konfigurace kvantizace (zůstává stejná, je nutná pro běh na vaší GPU) ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

print("✅ Konfigurace připravena.")
print(f"Bude se načítat originální model: {base_model_name}")

✅ Konfigurace připravena.
Bude se načítat originální model: mistralai/Mistral-7B-Instruct-v0.1


In [5]:
# Buňka 3 (OPRAVENÁ VERZE PRO ORIGINÁLNÍ MODEL)
print("⏳ Načítám originální model a tokenizer... Toto může trvat několik minut.")

try:
    # Načtení tokenizeru
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    print("...Tokenizer načten.")

    # ------ ZDE JE OPĚT KLÍČOVÁ OPRAVA ------
    # Explicitně řekneme, aby se pro GPU s ID 0 použilo maximum paměti
    # a pro CPU se nepoužívala žádná paměť na offloadování vrstev.
    # Použijeme stejný trik, který fungoval u fine-tuned modelu.
    max_memory_map = {0: "8GiB", "cpu": "0GiB"} 

    # Načtení kvantizovaného modelu s explicitním řízením paměti
    model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        quantization_config=bnb_config,
        device_map="auto",
        max_memory=max_memory_map, # Přidán osvědčený parametr
        torch_dtype=torch.bfloat16,
    )
    
    # Důležité: Přepneme model do evaluačního módu
    model.eval()
    
    print("\n✅ Originální model je načten a připraven k chatu!")

except Exception as e:
    import traceback
    print("\n❌ CHYBA PŘI NAČÍTÁNÍ MODELU!")
    traceback.print_exc()

⏳ Načítám originální model a tokenizer... Toto může trvat několik minut.
...Tokenizer načten.


Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.12s/it]



✅ Originální model je načten a připraven k chatu!


In [6]:
# Buňka 4: Interaktivní chat s originálním modelem

# Seznam pro uchování historie konverzace
chat_history = []

print("✅ Chat s ORIGINÁLNÍM modelem je připraven! Můžete začít psát.")
print("Napište 'konec' nebo 'exit' pro ukončení chatu.")
print("-" * 50)

while True:
    # 1. Získání vstupu od uživatele
    user_input = input("Vy: ")
    if user_input.lower() in ["konec", "exit", "quit"]:
        print("Chat ukončen.")
        break
        
    # 2. Sestavení promptu s historií (formát pro Mistral-Instruct)
    history_prompt_parts = []
    for user_msg, assistant_msg in chat_history:
        history_prompt_parts.append(f"[INST] {user_msg} [/INST] {assistant_msg}</s>")
    history_prompt = "".join(history_prompt_parts)
    
    final_prompt = f"<s>{history_prompt}[INST] {user_input} [/INST]"
    
    # 3. Tokenizace a příprava vstupu
    inputs = tokenizer(final_prompt, return_tensors="pt").to(model.device)
    input_token_len = inputs.input_ids.shape[1]
    
    print("\nAsistent přemýšlí...")
    
    # 4. Generování odpovědi
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1024,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    # 5. Dekódování a vyčištění odpovědi
    generated_tokens = outputs[0, input_token_len:]
    clean_response = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
    
    print(f"\nAI Asistent (Originál): {clean_response}")
    print("-" * 50)
    
    # 6. Aktualizace historie konverzace
    chat_history.append((user_input, clean_response))

✅ Chat s ORIGINÁLNÍM modelem je připraven! Můžete začít psát.
Napište 'konec' nebo 'exit' pro ukončení chatu.
--------------------------------------------------

Asistent přemýšlí...

AI Asistent (Originál): In quantum mechanics, a quantum state is a mathematical representation of the current state or condition of a quantum system. It encodes all the information about the system that can be known through measurements, including its energy, momentum, spin, and other observable properties.

Quantum states are typically represented using mathematical equations called wave functions, which describe the probability distribution of particles in space and time. The wave function contains all the information about the system's behavior that can be observed experimentally, including its energy, momentum, and spin.

The concept of quantum states is fundamental to quantum mechanics, as it allows us to understand the behavior of matter and energy at the atomic and subatomic level. Quantum states p

KeyboardInterrupt: Interrupted by user